In [1]:
# ignore System Warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
from crewai import Crew, Task, Agent

In [3]:
from dotenv import load_dotenv

load_dotenv()

# # Importing the necessary libraries
# import os

True

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [6]:
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify & book an appropiate venue base on event requirements",
  backstory=(
    "With a keep sense of space & Understanding of event logistics,"
    "you excel at finding & securing the perfect venue that fits the event's theme,"
    "size & budget constraints."
  ),
  tools=[search_tool, scrape_tool],
  verbose=True
)

In [8]:
logistics_manager = Agent(
      role="Logistics Manager",
  goal="Manage all logistics for the event including catering & equipment",
  backstory=(
    "Organized & detail-oriented, you ensure that every logistical aspect of the event"
    "from catering to equipment setup is flawlessly to create a seamless experience."
  ),
  tools=[search_tool, scrape_tool],
  verbose=True
)

In [9]:
marketing_communications_agent=Agent(
    role="Marketing & Communications Agent",
    goal="Effectively market the event & communicate with participants",
    backstory=(
    "Creative & communicative, you craft compelling messages &"
    "engage with potential attendees to maximize event exposure & participation."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True
)

# Creating Venue Pydantic Object

In [11]:
from pydantic import BaseModel

class VenueDetails(BaseModel):
    name:str
    address:str
    capacity: int
    booking_status: str

In [12]:
venue_task = Task(
    description="Find a {venue_type} venue in {event_city} that meets criteria for {event_topic}/{event_description}.",
  expected_output="All the details of a specifically choosen venue you found to accommodate the event.",
  human_input=True,
  output_json=VenueDetails,
  output_file="venue_details.json",
  agent=venue_coordinator
)

In [16]:
logistics_task= Task(
   description="Coordinate catering & equipment for an event with {expected_participants} participants on {tentative_date}.",
  expected_output="Confirmation of all logistics arrangments including catering & equipment setup",
  human_input=True,
  async_execution=False,
  agent=logistics_manager 
)

In [17]:
marketing_task = Task(
    description="Promote the {event_topic}/{event_description} aiming to engage at least {expected_participants} potential attendees.",
  expected_output="Report on marketing activities & attendee engagement formatted as markdown.",
  async_execution=True,
  output_file="marketing_report.md",
  agent=marketing_communications_agent
)

In [18]:
event_managment_crew = Crew(
    agents=[venue_coordinator, logistics_manager, marketing_communications_agent],
    tasks=[venue_task, logistics_task, marketing_task],
    verbose=True
)

2025-02-13 08:15:43,876 - 140704382634752 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [19]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "Karachi, Pakistan",
    'tentative_date': "2025-03-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [20]:
result = event_managment_crew.kickoff(inputs=event_details)

# Agent: Venue Coordinator
## Task: Find a Conference Hall venue in Karachi, Pakistan that meets criteria for Tech Innovation Conference/A gathering of tech innovators and industry leaders to explore future technologies..
 

I encountered an error while trying to use the tool. This was the error: 'SERPER_API_KEY'.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



# Agent: Venue Coordinator
## Thought: To find a suitable conference hall venue in Karachi, Pakistan that fits the requirements for a Tech Innovation Conference, I should search for venues that specialize in technology-themed events and can accommodate a large number of attendees, along with having th

In [21]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'University Road, Gulshan-e-Iqbal, Karachi, Pakistan',
 'booking_status': 'Requires confirmation via venue contact',
 'capacity': 5000,
 'name': 'Karachi Expo Center'}


In [22]:
from IPython.display import Markdown

Markdown(result.raw)

# Tech Innovation Conference Marketing & Engagement Report

## Objective:
Promote the Tech Innovation Conference, a gathering of tech innovators and industry leaders to explore future technologies, with a goal of engaging at least 500 potential attendees.

## Strategies:

### 1. Digital Marketing Campaigns
- **Social Media Marketing**: Utilize platforms like LinkedIn, Twitter, and Facebook to create event pages, share innovation-related content, feature speaker highlights, and create countdowns to the event.
- **Influencer Partnerships**: Collaborate with tech industry influencers to reach a wider audience. Influencers can share event details with their followers, conduct Q&A sessions, or even speak at the conference.
- **Email Marketing**: Send out newsletters and personalized invites to previous attendees, tech industry professionals, and other potential participants. Use engaging subject lines and informative content about what they can expect at the conference.

### 2. Content Marketing
- **Blog Posts & Articles**: Publish content about the latest tech trends that will be discussed at the conference, interviews with key speakers, and past conference success stories on the event website and partner blogs.
- **Webinars and Pre-event Meetups**: Host online sessions leading up to the conference with industry leaders as a teaser to spark interest and encourage registrations.

### 3. Partnerships and Sponsorships
- **Corporate Partnerships**: Partner with tech companies and startups for mutual promotion. They can offer promotions to their customer base in exchange for branding opportunities at the event.
- **Media Partnerships**: Work with tech-focused publications and websites to feature the conference in articles, advertisements, or newsletters.

### 4. Incentives for Early Registration
- **Discounts & Promotions**: Offer discounted tickets at tiered pricing for those who register early. Bundle packages can include additional perks such as exclusive networking events access.
- **Referral Programs**: Encourage previous attendees to bring colleagues by offering them discounts or special access passes.

## Engagement Tactics:
- **Interactive Platforms**: Use event apps to keep attendees engaged with live Q&A sessions, interactive sessions, and event agenda updates. 
- **Gamification**: Introduce challenges or scavenger hunts through the app to incentivize attendee interaction.
- **Networking Opportunities**: Create spaces and opportunities for attendees to network virtually and in-person before and during the event.

## Monitoring and Feedback:
- **Event Analytics**: Use analytics tools to measure engagement levels across digital platforms and adjust marketing strategies accordingly.
- **Post-event Surveys**: Collect feedback through surveys to understand attendee experience and areas for improvement in future events.

## Conclusion:
By effectively utilizing a blend of digital marketing, strategic partnerships, and engagement incentives, the Tech Innovation Conference aims to surpass the engagement goal, fostering a vibrant environment for tech innovation and networking.
```